In [4]:
import numpy as np
import random
from collections import Counter
import pandas as pd

# Define the sequences and conditions
sequences = [0, 1, 2, 3]
conditions = [0, 1]
nTr = 65 # number of trials
def generate_random_sequence(numbers_list, seq_rep, seed=None):
    # Seed the random number generator if a seed is provided
    if seed is not None:
        random.seed(seed)

    # Create a sequence by sampling from the numbers list
    random_sequence = []
    for i in range(seq_rep):
        random_sequence.append(random.sample(numbers_list, len(numbers_list)))

    return np.concatenate(random_sequence)

# Define the list of numbers
numbers_list = [1, 2, 3, 4]
numbers_list = [0, 1]

# Define the desired length of the sequence
sequence_length = 10

# Generate the random sequence
random_sequence = generate_random_sequence(numbers_list, sequence_length)
print(random_sequence)


[0 1 0 1 0 1 0 1 0 1 1 0 1 0 0 1 0 1 1 0]


In [6]:
## Final solution for generating an Euler cycle

import random

def shuffle(array):
    currentIndex = len(array)
    while currentIndex > 0:
        randomIndex = random.randint(0, currentIndex - 1)
        currentIndex -= 1
        array[currentIndex], array[randomIndex] = array[randomIndex], array[currentIndex]
    return array

def generate_trial_states(init_seq):
    # args- init_seq: int, the initial sequence number
    seq_cond = np.array([[0,0],[0,1],[1,0],[1,1],[2,0],[2,1],[3,0],[3,1]])
    while True: 
        seq = []
        while len(seq) < 65: 
            seq.clear()
            numbers = list(range(8))
            paths = [shuffle([[a, b] for b in numbers]) for a in numbers]
            if len(seq)==0:
                currNum = init_seq
            else:
                currNum = random.randint(0, 7)
            while True:
                seq.append(currNum)

                if len(paths[currNum - 1]) <= 0:
                    break

                path = paths[currNum - 1].pop()

                if len(paths[currNum - 1]) >= 1 and len(paths[path[1] - 1]) <= 0:
                    continue
                currNum = path[1]
        trial_states = seq_cond[np.array(seq)]
        trial_transitions = [(trial_states[i], trial_states[i + 1]) for i in range(len(trial_states) - 1)]
        # Calculate the number of unique transitions
        trial_transitions = [tuple(map(tuple, transition)) for transition in trial_transitions]
        unique_transitions = set(trial_transitions)
        num_unique_transitions = len(unique_transitions)
        if num_unique_transitions==64:
            break
        
    return trial_states, seq

    # print(paths)


# print(len(seq))
# print(seq)


In [8]:
## Final solution for generating an Euler cycle

import random
import numpy as np

def shuffle(array):
    # Function to shuffle elements of an array
    currentIndex = len(array)
    while currentIndex > 0:
        # Pick a random index
        randomIndex = random.randint(0, currentIndex - 1)
        currentIndex -= 1
        # Swap the current element with the random element
        array[currentIndex], array[randomIndex] = array[randomIndex], array[currentIndex]
    return array

def generate_trial_states(init_seq):
    # args- init_seq: int, the initial sequence number

    # Define 8 trials states (sequence ID, Cue type, 0: letter, 1: spatial)
    state_mapping = np.array([[0,0],[0,1],[1,0],[1,1],[2,0],[2,1],[3,0],[3,1]])
    # Infinite loop until a valid sequence is generated
    while True: 
        # Initialize an empty sequence
        seq = []
        # Generate a sequence of states
        while len(seq) < 65: # Make sure that length of the sequence is 65
            seq.clear()
            numbers = list(range(8))
            # Generate random paths between states and shuffle them
            paths = [shuffle([[a, b] for b in numbers]) for a in numbers]
            # Choose initial state based on input argument
            if len(seq) == 0:
                currNum = init_seq
            else:
                currNum = random.randint(0, 7)
            # Traverse the graph until all states are visited
            while True:
                seq.append(currNum)
                # If there are no more paths from current state, break the loop
                if len(paths[currNum - 1]) <= 0:
                    break
                # Choose a random path from the current state and move to the next state
                path = paths[currNum - 1].pop()
                # If there are more than one path from current state and no path from the next state, continue
                if len(paths[currNum - 1]) >= 1 and len(paths[path[1] - 1]) <= 0:
                    continue
                currNum = path[1]
        # Convert the sequence of states into binary sequences
        trial_states = state_mapping[np.array(seq)]
        # Generate transitions between states
        trial_transitions = [(trial_states[i], trial_states[i + 1]) for i in range(len(trial_states) - 1)]
        # Convert transitions to tuples of tuples for uniqueness check
        trial_transitions = [tuple(map(tuple, transition)) for transition in trial_transitions]
        # Check the number of unique transitions
        unique_transitions = set(trial_transitions)
        num_unique_transitions = len(unique_transitions)
        # If there are exactly 64 unique transitions, break the loop and return the sequence
        if num_unique_transitions == 64:
            break
        
    return trial_states, seq


In [11]:
def generate_tgt_file_fmri(subj_id, run_id, trial_states):
    # trial_states[k][0]: sequence ID (1,2,3,4), 
    # trial_states[k][1]: condition (Number cue: 0, Spatial visual cue: 1)
    # motor_sequences = [[3, 2, 4, 5, 2, 1, 5, 1, 4],[3, 5, 1, 3, 2, 4, 5, 2, 1], [1, 3, 2, 1, 2, 4, 5, 1, 3], [1, 2, 4, 3, 2, 4, 3, 5, 1]]
    motor_sequences = [[3, 2, 4, 5, 1],[3, 5, 1, 2, 4], [1, 3, 2, 5, 4], [1, 4, 5, 2, 3]]

    M = np.size(motor_sequences,1) # length of motor sequence
    # Set constant values
    # stim_time_value = 15000
    prep_time_value = 1000
    mov_time_lim_value = 3000
    startTime = 8020 # about 8 sec after a triggering signal
    # feedback_value = 1

    # Generate 27 trials
    data = []
    n_horizon = len(motor_sequences[0])
    nTr = len(trial_states)+3 # number of trials = 65, but will add 3 additional trials due to resting periods
    k = 0
    for m in range(nTr):        
        # Create a row of data
        if (m+1)%17 == 0:  # For the last trial of each block consisting of 16 trials, not the first trial
            iti_value = 16000 # 10 s ITI for baseline activity estimation
            row = [startTime, trial_states[k][1]] + motor_sequences[trial_states[k][0]] + [''.join(map(str, motor_sequences[trial_states[k][0]])), iti_value, n_horizon, prep_time_value, mov_time_lim_value]
            k = k+1
        elif (m+1)%17 == 1 and m!=0:# For the last trial of each block consisting of 16 trials, not the first trial
            iti_value = 1000 # 10 s ITI for baseline activity estimation
            row = [startTime, trial_states[k-1][1]] + motor_sequences[trial_states[k-1][0]] + [''.join(map(str, motor_sequences[trial_states[k-1][0]])), iti_value, n_horizon, prep_time_value, mov_time_lim_value]
        else:
            iti_value = 1000 # 1 s ITI for other trials
            row = [startTime, trial_states[k][1]] + motor_sequences[trial_states[k][0]] + [''.join(map(str, motor_sequences[trial_states[k][0]])), iti_value, n_horizon, prep_time_value, mov_time_lim_value]
            k = k+1

        # row = [seq_type, feedback_value] + random_numbers + [''.join(map(str, random_numbers)), iti_value, random_horizons[k], stim_time_value, prep_time_value]

        # Append the row to the data list
        data.append(row)
        startTime = startTime + prep_time_value + mov_time_lim_value + iti_value

    # Create a DataFrame using pandas
    columns = ["startTime", "seqType"] + [f"press{i}" for i in range(1, M+1)] + ["cueP", "iti", "Horizon", "PrepTime", 'MovTimeLim']
    df = pd.DataFrame(data, columns=columns)

    # Save the DataFrame to a CSV file with .tgt extension
    run_id = str(run_id).zfill(2)
    subj_id = str(subj_id).zfill(2)
    filename = f"fmri_ssh{subj_id}_r{run_id}.tgt"
  #  filename = "fmri_ssh00_r00.tgt"
    df.to_csv(filename, sep='\t',  index=False)


In [13]:
seq_cond = [(0,0),(0,1),(1,0),(1,1),(2,0),(2,1),(3,0),(3,1)]
init_seq_conds = np.random.permutation(8)

seq_cond = [(0,0),(0,1),(1,0),(1,1),(2,0),(2,1),(3,0),(3,1)]
matrices = []
for s in range(1,3):
    
    # Now the elements of the matrix are the indices of the seq_cond array and I need 3D array of seq_conds using for loop

    init_sequence = np.random.permutation(8)
    for r in range(1,9):
        trial_states = generate_trial_states(init_sequence[r-1])
        generate_tgt_file_fmri(s, r, trial_states)

In [ ]:


def generate_trial_states_OLD(init_seq_conds, nTr):
    nBlock = int(nTr/6)-1

    while True:
        # Initialize the list to store trial states
        trial_states = np.zeros((nTr, 2),dtype=int)

        # Set the initial condition for the first trial
        current_sequence = init_seq_conds[0][0]
        current_condition = init_cond  # G=1, V=1
        trial_states[0]= [current_sequence, current_condition]

        # # Generate the trial states
        # # rand_seq = generate_random_sequence(sequences, int(nTr/len(sequences)), seed=42)
        rand_cond = generate_random_sequence(conditions, int(nTr/len(conditions)))
        rand_bin = generate_random_sequence([0, 1], int(nTr/2))
        rep_flag_seq = 1
        rep_flag_cond = 1
        max_rep = 3
        for k in range(nTr-1):  
            # 
            # if rand_bin[k] == 0:
            if ((random.random() < 0.5) and (rep_flag_seq < max_rep)):
                next_sequence = current_sequence
                rep_flag_seq += 1
            else: # randomly choose from the remaining sequences
                # Decide the probability of choosing each sequence depending on its frequency so far, sum of prob_seq should be equal to 1
                other_seqs = [seq for seq in sequences if seq != current_sequence]
                # temp = np.array([np.count_nonzero(trial_states[:k+1,0] == elem) for elem in other_seqs])
                # if any(temp>0):
                #     temp = sum(temp)-temp
                #     prob_seq = temp/sum(temp) 
                # else:
                #     prob_seq = [1/len(other_seqs)]*len(other_seqs)
                # next_sequence = np.random.choice(other_seqs, p = prob_seq)
                next_sequence = np.random.choice(other_seqs)
                rep_flag_seq = 1
            # if rand_cond[k] ==0:
            if ((random.random() < 0.5) and (rep_flag_cond < max_rep)):
                next_condition = current_condition
                rep_flag_cond += 1
            else: # choose the other condition
                next_condition = 1 - current_condition
                rep_flag_cond = 1
            trial_states[k+1] = [next_sequence, next_condition]
            current_sequence = next_sequence
            current_condition = next_condition

        # Calculate frequency of trial transitions from two consecutive trials
        trial_transitions = [(trial_states[i], trial_states[i + 1]) for i in range(len(trial_states) - 1)]

        # Convert NumPy arrays to tuples
        trial_transitions = [tuple(map(tuple, transition)) for transition in trial_transitions]

        # Calculate the number of unique transitions
        unique_transitions = set(trial_transitions)
        num_unique_transitions = len(unique_transitions)

        count_all = []
        for transition, count in Counter(trial_transitions).items():
            count_all.append(count)
        # Define counters for each case
        no_change_count = 0
        sequence_change_count = 0
        condition_change_count = 0
        both_change_count = 0
        # Iterate through trial transitions
        # for transition in trial_transitions:
        #     current_state, next_state = transition
        for blk in range(6): # 6 blocks, each block consisting of 10 trials is defined between 10-s rest periods
            for k in range(nBlock):
                current_state, next_state = trial_states[blk*nBlock+k], trial_states[blk*nBlock+k+1]
                # Check if the sequence and condition remain unchanged
                if current_state[0] == next_state[0] and current_state[1] == next_state[1]:
                    no_change_count += 1
                # Check if only the sequence changes
                elif current_state[0] != next_state[0] and current_state[1] == next_state[1]:
                    sequence_change_count += 1
                # Check if only the condition changes
                elif current_state[0] == next_state[0] and current_state[1] != next_state[1]:
                    condition_change_count += 1
                # Check if both the sequence and condition change
                elif current_state[0] != next_state[0] and current_state[1] != next_state[1]:
                    both_change_count += 1
        
        # If we have found more than 36 unique transitions and largest value in count_all is less than 5, break the loop
        # no_consecutive = has_not_consecutive_same(trial_states_array)
        # no_consecutive_cond = has_not_consecutive_same_condition(trial_states_array)
        # no_consecutive_seq = has_not_consecutive_same_sequence(trial_states_array)
        #if (min(Counter(trial_states[:,1]).values())>0.45*nTr) & (max(Counter(trial_states[:,1]).values())<0.55*nTr) & (max(Counter(trial_states[:,0]).values())<0.3*nTr) & (min(Counter(trial_states[:,0]).values())>0.2*nTr) & (num_unique_transitions > 10) & (max(count_all)<5) & (max(no_change_count, sequence_change_count, condition_change_count, both_change_count) < 0.3*nTr) & (min(no_change_count, sequence_change_count, condition_change_count, both_change_count) > 0.2*nTr):
        if (min(Counter(trial_states[:,1]).values())>0.45*nTr) & (max(Counter(trial_states[:,1]).values())<0.55*nTr) & (max(Counter(trial_states[:,0]).values())<0.3*nTr) & (min(Counter(trial_states[:,0]).values())>0.2*nTr) & (num_unique_transitions > 10) & (max(count_all)<5) & (max(no_change_count, sequence_change_count, condition_change_count, both_change_count) == 15) & (min(no_change_count, sequence_change_count, condition_change_count, both_change_count) == 15):

            break

    # # Print the frequency of trial transitions
    # print("Frequency of Trial Transitions:")
    # for transition, count in Counter(trial_transitions).items():
    #     print(f"Transition: {transition}, Frequency: {count}")
    
    print(f"Number of unique transitions: {num_unique_transitions}")
    print(no_change_count, sequence_change_count, condition_change_count, both_change_count)
    print(f"Number of sequences: {Counter(trial_states[:,0])}")
    print(f"Number of conditions: {Counter(trial_states[:,1])}")
    return trial_states
    # # Define the file name
    # file_name = "trial_states.csv"

    # # Save the trial_states_array as a CSV file
    # np.savetxt(file_name, trial_states_array, delimiter=",", fmt="%s")
